In [1]:
import matplotlib.pyplot as pyplt
import pandas as pd
import time
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
import datetime
from collections import OrderedDict



In [2]:
import src.constants as constants
import src.filenameGenerator as fileNameGen
import src.imageLoader as iLoader



In [3]:
"""""""""""""""""""""""
" Lecture fichiers CSV
"""""""""""""""""""""""
# Le fichier CSV contenant les données galaxies
csv_vectors = 'data/processed/galaxy_feature_vectors.csv'

# Recupere les données galaxies (saute la premiere et derniere colonne --> ID galaxie)
X_galaxy = pd.read_csv(csv_vectors, header=None).values[:,1:-1]

# Recupere la colonne indiquant si c'est une galaxie spiral (1) ou smooth (0)
Y_galaxy = pd.read_csv(csv_vectors, header=None).values[:,-1:].astype(int).flatten()



In [4]:
"""""""""""""""""""""""
" Normalisation
"""""""""""""""""""""""

scaler = StandardScaler()
X_galaxy = scaler.fit_transform(X_galaxy)

In [5]:
# One Hot encode les étiquettes

gala_labels_2d = Y_galaxy.reshape(len(Y_galaxy))

oh_encoder = OneHotEncoder(sparse=False)
gala_labels_2d = gala_labels_2d.reshape(-1,1)
gala_labels_oh = oh_encoder.fit_transform(gala_labels_2d)



C:\Users\Jonathan\Anaconda3\envs\gti770\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [6]:
def executeNN(x, y, name, lr, epoch, layers):
    tk_board = TensorBoard(log_dir=constants.LOGS_PATH + name)

    model = keras.Sequential()
    model.add(keras.layers.Dense(layers[0], input_shape=(x.shape[1],), activation='relu'))
    for index in range(1, len(layers)-1):
        model.add(keras.layers.Dense(layers[index], activation='relu'))
    model.add(keras.layers.Dense(layers[len(layers)-1], activation='softmax'))
    
    optimizer = keras.optimizers.RMSprop(lr=lr)
    model.compile(
        optimizer=optimizer, 
        loss='categorical_crossentropy', 
        metrics=[
            tf.keras.metrics.Accuracy(), 
            tf.keras.metrics.Precision(name='precision'), 
            tf.keras.metrics.Recall(name='recall')
        ]
    )

    model.summary()

    start_train_time = time.perf_counter()
    history = model.fit(x=x, y=y, batch_size=100, epochs=epoch, shuffle=True, validation_split=0.2, callbacks=[tk_board])
    train_time = time.perf_counter() - start_train_time

    history = history.history
    index = len(history['precision']) -1
    precision = history['precision'][index]
    recall = history['recall'][index]
    val_precision = history['val_precision'][index]
    val_recall = history['val_recall'][index]
    f1 = 2 * (precision * recall) / (precision + recall)
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall)
    accuracy = history['accuracy'][index]
    val_accuracy = history['val_accuracy'][index]

    run_results = OrderedDict([
        ('run_name', name),
        ('train_time (seconds)', train_time),
        ('learning rate', lr),
        ('epoch', epoch),
        ('accuracy', accuracy),
        ('val_accuracy', val_accuracy),
        ('f1', f1),
        ('val_f1', val_f1)
    ])

    for index in range(0, len(layers)):
        run_results['layer: ' + str(index)] = layers[index]

    return run_results


In [7]:
results = []
x = X_galaxy
y = gala_labels_oh

In [8]:
"""""""""""""""""""""""
" Baseline
"""""""""""""""""""""""
results.append(executeNN(x, y, "NN/BaseLine",0.005,60, [100,100,2]))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               7900      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 18,202
Trainable params: 18,202
Non-trainable params: 0
_________________________________________________________________
Train on 13526 samples, validate on 3382 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/60
13526/13526 [==============================] - 0s 35us/sample - loss: 0.1826 - accuracy: 4.8056e-04 - precision: 0.9326 - recall: 0.9326 - val_loss: 0.1646 - val_accuracy: 4.4352e-04 - val_precision: 0.9474 - val_recall: 0

In [9]:
"""""""""""""""""""""""
" Couches
"""""""""""""""""""""""
results.append(executeNN(x, y, "NN/CouchePlus",0.005,60, [100, 100, 100, 100,2]))
results.append(executeNN(x, y, "NN/CoucheMoins",0.005,60, [100,2]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               7900      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 202       
Total params: 38,402
Trainable params: 38,402
Non-trainable params: 0
_________________________________________________________________
Train on 13526 samples, validate on 3382 samples
Epoch 1/60
13526/13526 [==============================] - 1s 43us/sample - loss: 0.2129 -

Epoch 32/60
13526/13526 [==============================] - 0s 19us/sample - loss: 0.0615 - accuracy: 0.5368 - precision: 0.9831 - recall: 0.9831 - val_loss: 0.2494 - val_accuracy: 0.6066 - val_precision: 0.9636 - val_recall: 0.9636
Epoch 33/60
13526/13526 [==============================] - 0s 19us/sample - loss: 0.0624 - accuracy: 0.5757 - precision: 0.9838 - recall: 0.9838 - val_loss: 0.2460 - val_accuracy: 0.4559 - val_precision: 0.9539 - val_recall: 0.9539
Epoch 34/60
13526/13526 [==============================] - 0s 19us/sample - loss: 0.0616 - accuracy: 0.5502 - precision: 0.9825 - recall: 0.9825 - val_loss: 0.2267 - val_accuracy: 0.4939 - val_precision: 0.9607 - val_recall: 0.9607
Epoch 35/60
13526/13526 [==============================] - 0s 18us/sample - loss: 0.0570 - accuracy: 0.5511 - precision: 0.9842 - recall: 0.9842 - val_loss: 0.2894 - val_accuracy: 0.5911 - val_precision: 0.9622 - val_recall: 0.9622
Epoch 36/60
13526/13526 [==============================] - 0s 19us/sampl

13526/13526 [==============================] - 0s 15us/sample - loss: 0.0514 - accuracy: 0.1579 - precision: 0.9853 - recall: 0.9853 - val_loss: 0.2078 - val_accuracy: 0.1674 - val_precision: 0.9574 - val_recall: 0.9574
Epoch 40/60
13526/13526 [==============================] - 0s 16us/sample - loss: 0.0501 - accuracy: 0.1635 - precision: 0.9866 - recall: 0.9866 - val_loss: 0.2110 - val_accuracy: 0.1875 - val_precision: 0.9556 - val_recall: 0.9556
Epoch 41/60
13526/13526 [==============================] - 0s 14us/sample - loss: 0.0486 - accuracy: 0.1746 - precision: 0.9853 - recall: 0.9853 - val_loss: 0.1865 - val_accuracy: 0.1616 - val_precision: 0.9624 - val_recall: 0.9624
Epoch 42/60
13526/13526 [==============================] - 0s 14us/sample - loss: 0.0464 - accuracy: 0.1757 - precision: 0.9868 - recall: 0.9868 - val_loss: 0.1907 - val_accuracy: 0.1731 - val_precision: 0.9604 - val_recall: 0.9604
Epoch 43/60
13526/13526 [==============================] - 0s 14us/sample - loss: 0.

In [10]:
"""""""""""""""""""""""
" Perceptron
"""""""""""""""""""""""
results.append(executeNN(x, y, "NN/PerceptronPlus",0.005,60, [500,500,2]))
results.append(executeNN(x, y, "NN/PerceptronMoins",0.005,60, [20,20,2]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 500)               39500     
_________________________________________________________________
dense_11 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 1002      
Total params: 291,002
Trainable params: 291,002
Non-trainable params: 0
_________________________________________________________________
Train on 13526 samples, validate on 3382 samples
Epoch 1/60
13526/13526 [==============================] - 1s 73us/sample - loss: 0.2920 - accuracy: 0.0113 - precision: 0.9209 - recall: 0.9209 - val_loss: 0.1615 - val_accuracy: 0.0089 - val_precision: 0.9459 - val_recall: 0.9459
Epoch 2/60
13526/13526 [==============================] - 1s 45us/sample - loss: 0.1590 - accuracy: 0.0132 - precision: 

Epoch 33/60
13526/13526 [==============================] - 1s 45us/sample - loss: 0.0720 - accuracy: 0.3955 - precision: 0.9840 - recall: 0.9840 - val_loss: 0.2694 - val_accuracy: 0.3890 - val_precision: 0.9619 - val_recall: 0.9619
Epoch 34/60
13526/13526 [==============================] - 1s 44us/sample - loss: 0.0759 - accuracy: 0.4095 - precision: 0.9851 - recall: 0.9851 - val_loss: 0.2610 - val_accuracy: 0.3862 - val_precision: 0.9622 - val_recall: 0.9622
Epoch 35/60
13526/13526 [==============================] - 1s 46us/sample - loss: 0.0734 - accuracy: 0.4245 - precision: 0.9859 - recall: 0.9859 - val_loss: 0.3063 - val_accuracy: 0.4554 - val_precision: 0.9607 - val_recall: 0.9607
Epoch 36/60
13526/13526 [==============================] - 1s 45us/sample - loss: 0.0726 - accuracy: 0.4195 - precision: 0.9838 - recall: 0.9838 - val_loss: 0.2943 - val_accuracy: 0.4336 - val_precision: 0.9604 - val_recall: 0.9604
Epoch 37/60
13526/13526 [==============================] - 1s 45us/sampl

Epoch 5/60
13526/13526 [==============================] - 0s 15us/sample - loss: 0.1031 - accuracy: 0.0026 - precision: 0.9613 - recall: 0.9613 - val_loss: 0.1350 - val_accuracy: 0.0022 - val_precision: 0.9562 - val_recall: 0.9562
Epoch 6/60
13526/13526 [==============================] - 0s 20us/sample - loss: 0.1011 - accuracy: 0.0023 - precision: 0.9643 - recall: 0.9643 - val_loss: 0.1141 - val_accuracy: 0.0019 - val_precision: 0.9639 - val_recall: 0.9639
Epoch 7/60
13526/13526 [==============================] - 0s 23us/sample - loss: 0.0940 - accuracy: 0.0028 - precision: 0.9655 - recall: 0.9655 - val_loss: 0.1243 - val_accuracy: 0.0022 - val_precision: 0.9583 - val_recall: 0.9583
Epoch 8/60
13526/13526 [==============================] - 0s 16us/sample - loss: 0.0912 - accuracy: 0.0043 - precision: 0.9664 - recall: 0.9664 - val_loss: 0.1164 - val_accuracy: 0.0075 - val_precision: 0.9601 - val_recall: 0.9601
Epoch 9/60
13526/13526 [==============================] - 0s 16us/sample - l

Epoch 40/60
13526/13526 [==============================] - 0s 13us/sample - loss: 0.0526 - accuracy: 0.0950 - precision: 0.9809 - recall: 0.9809 - val_loss: 0.1583 - val_accuracy: 0.1082 - val_precision: 0.9624 - val_recall: 0.9624
Epoch 41/60
13526/13526 [==============================] - 0s 13us/sample - loss: 0.0493 - accuracy: 0.1069 - precision: 0.9828 - recall: 0.9828 - val_loss: 0.1550 - val_accuracy: 0.0988 - val_precision: 0.9610 - val_recall: 0.9610
Epoch 42/60
13526/13526 [==============================] - 0s 12us/sample - loss: 0.0526 - accuracy: 0.1106 - precision: 0.9823 - recall: 0.9823 - val_loss: 0.1435 - val_accuracy: 0.0940 - val_precision: 0.9645 - val_recall: 0.9645
Epoch 43/60
13526/13526 [==============================] - 0s 12us/sample - loss: 0.0494 - accuracy: 0.1068 - precision: 0.9817 - recall: 0.9817 - val_loss: 0.1729 - val_accuracy: 0.1168 - val_precision: 0.9610 - val_recall: 0.9610
Epoch 44/60
13526/13526 [==============================] - 0s 13us/sampl

In [11]:
"""""""""""""""""""""""
" Epoch
"""""""""""""""""""""""
results.append(executeNN(x, y, "NN/EpochPlus",0.005,200, [100,100,2]))
results.append(executeNN(x, y, "NN/EpochMoins",0.005,20, [100,100,2]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 100)               7900      
_________________________________________________________________
dense_17 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 202       
Total params: 18,202
Trainable params: 18,202
Non-trainable params: 0
_________________________________________________________________
Train on 13526 samples, validate on 3382 samples
Epoch 1/200
13526/13526 [==============================] - 1s 42us/sample - loss: 0.1922 - accuracy: 0.0013 - precision: 0.9292 - recall: 0.9292 - val_loss: 0.1568 - val_accuracy: 0.0012 - val_precision: 0.9432 - val_recall: 0.9432
Epoch 2/200
13526/13526 [==============================] - 0s 18us/sample - loss: 0.1326 - accuracy: 0.0017 - precision: 

Epoch 33/200
13526/13526 [==============================] - 0s 15us/sample - loss: 0.0451 - accuracy: 0.3416 - precision: 0.9882 - recall: 0.9882 - val_loss: 0.2483 - val_accuracy: 0.3278 - val_precision: 0.9610 - val_recall: 0.9610
Epoch 34/200
13526/13526 [==============================] - 0s 15us/sample - loss: 0.0407 - accuracy: 0.3388 - precision: 0.9902 - recall: 0.9902 - val_loss: 0.2176 - val_accuracy: 0.2960 - val_precision: 0.9624 - val_recall: 0.9624
Epoch 35/200
13526/13526 [==============================] - 0s 16us/sample - loss: 0.0456 - accuracy: 0.3478 - precision: 0.9895 - recall: 0.9895 - val_loss: 0.2380 - val_accuracy: 0.3084 - val_precision: 0.9586 - val_recall: 0.9586
Epoch 36/200
13526/13526 [==============================] - 0s 16us/sample - loss: 0.0412 - accuracy: 0.3656 - precision: 0.9913 - recall: 0.9913 - val_loss: 0.2827 - val_accuracy: 0.3554 - val_precision: 0.9610 - val_recall: 0.9610
Epoch 37/200
13526/13526 [==============================] - 0s 16us/

13526/13526 [==============================] - 0s 20us/sample - loss: 0.0286 - accuracy: 0.6322 - precision: 0.9962 - recall: 0.9962 - val_loss: 0.4445 - val_accuracy: 0.6245 - val_precision: 0.9607 - val_recall: 0.9607
Epoch 104/200
13526/13526 [==============================] - 0s 20us/sample - loss: 0.0261 - accuracy: 0.6517 - precision: 0.9957 - recall: 0.9957 - val_loss: 0.4564 - val_accuracy: 0.6322 - val_precision: 0.9610 - val_recall: 0.9610
Epoch 105/200
13526/13526 [==============================] - 0s 20us/sample - loss: 0.0372 - accuracy: 0.6361 - precision: 0.9956 - recall: 0.9956 - val_loss: 0.4423 - val_accuracy: 0.6060 - val_precision: 0.9595 - val_recall: 0.9595
Epoch 106/200
13526/13526 [==============================] - 0s 20us/sample - loss: 0.0246 - accuracy: 0.6344 - precision: 0.9964 - recall: 0.9964 - val_loss: 0.4567 - val_accuracy: 0.6359 - val_precision: 0.9607 - val_recall: 0.9607
Epoch 107/200
13526/13526 [==============================] - 0s 19us/sample - 

13526/13526 [==============================] - 0s 19us/sample - loss: 0.0189 - accuracy: 0.7446 - precision: 0.9980 - recall: 0.9980 - val_loss: 0.5414 - val_accuracy: 0.7234 - val_precision: 0.9571 - val_recall: 0.9571
Epoch 174/200
13526/13526 [==============================] - 0s 19us/sample - loss: 0.0241 - accuracy: 0.7621 - precision: 0.9975 - recall: 0.9975 - val_loss: 0.5113 - val_accuracy: 0.7475 - val_precision: 0.9589 - val_recall: 0.9589
Epoch 175/200
13526/13526 [==============================] - 0s 19us/sample - loss: 0.0374 - accuracy: 0.7595 - precision: 0.9963 - recall: 0.9963 - val_loss: 0.4990 - val_accuracy: 0.7244 - val_precision: 0.9595 - val_recall: 0.9595
Epoch 176/200
13526/13526 [==============================] - 0s 21us/sample - loss: 0.0270 - accuracy: 0.7449 - precision: 0.9963 - recall: 0.9963 - val_loss: 0.5020 - val_accuracy: 0.7167 - val_precision: 0.9545 - val_recall: 0.9545
Epoch 177/200
13526/13526 [==============================] - 0s 19us/sample - 

Epoch 5/20
13526/13526 [==============================] - 0s 19us/sample - loss: 0.0987 - accuracy: 0.0076 - precision: 0.9664 - recall: 0.9664 - val_loss: 0.1338 - val_accuracy: 0.0129 - val_precision: 0.9592 - val_recall: 0.9592
Epoch 6/20
13526/13526 [==============================] - 0s 21us/sample - loss: 0.0944 - accuracy: 0.0157 - precision: 0.9658 - recall: 0.9658 - val_loss: 0.1351 - val_accuracy: 8.8705e-04 - val_precision: 0.9512 - val_recall: 0.9512
Epoch 7/20
13526/13526 [==============================] - 0s 19us/sample - loss: 0.0883 - accuracy: 0.0167 - precision: 0.9690 - recall: 0.9690 - val_loss: 0.1362 - val_accuracy: 0.0170 - val_precision: 0.9568 - val_recall: 0.9568
Epoch 8/20
13526/13526 [==============================] - 0s 17us/sample - loss: 0.0865 - accuracy: 0.0236 - precision: 0.9704 - recall: 0.9704 - val_loss: 0.1190 - val_accuracy: 0.0109 - val_precision: 0.9613 - val_recall: 0.9613
Epoch 9/20
13526/13526 [==============================] - 0s 17us/sample

In [12]:
"""""""""""""""""""""""
" Learning rate
"""""""""""""""""""""""
results.append(executeNN(x, y, "NN/LearningRatePlus",0.010,60, [100,100,2]))
results.append(executeNN(x, y, "NN/LearningRateMoins",0.0025,60, [100,100,2]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 100)               7900      
_________________________________________________________________
dense_23 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_24 (Dense)             (None, 2)                 202       
Total params: 18,202
Trainable params: 18,202
Non-trainable params: 0
_________________________________________________________________
Train on 13526 samples, validate on 3382 samples
Epoch 1/60
13526/13526 [==============================] - 1s 44us/sample - loss: 0.2163 - accuracy: 0.0012 - precision: 0.9243 - recall: 0.9243 - val_loss: 0.1948 - val_accuracy: 0.0024 - val_precision: 0.9391 - val_recall: 0.9391
Epoch 2/60
13526/13526 [==============================] - 0s 17us/sample - loss: 0.1398 - accuracy: 0.0059 - precision: 0.

Epoch 33/60
13526/13526 [==============================] - 0s 16us/sample - loss: 0.0705 - accuracy: 0.2949 - precision: 0.9814 - recall: 0.9814 - val_loss: 0.2433 - val_accuracy: 0.3026 - val_precision: 0.9568 - val_recall: 0.9568
Epoch 34/60
13526/13526 [==============================] - 0s 16us/sample - loss: 0.0808 - accuracy: 0.3214 - precision: 0.9815 - recall: 0.9815 - val_loss: 0.2544 - val_accuracy: 0.3128 - val_precision: 0.9577 - val_recall: 0.9577
Epoch 35/60
13526/13526 [==============================] - 0s 16us/sample - loss: 0.0615 - accuracy: 0.3216 - precision: 0.9837 - recall: 0.9837 - val_loss: 0.2732 - val_accuracy: 0.3216 - val_precision: 0.9577 - val_recall: 0.9577
Epoch 36/60
13526/13526 [==============================] - 0s 16us/sample - loss: 0.0720 - accuracy: 0.3450 - precision: 0.9830 - recall: 0.9830 - val_loss: 0.3323 - val_accuracy: 0.4110 - val_precision: 0.9559 - val_recall: 0.9559
Epoch 37/60
13526/13526 [==============================] - 0s 16us/sampl

Epoch 5/60
13526/13526 [==============================] - 0s 16us/sample - loss: 0.0986 - accuracy: 0.0031 - precision: 0.9668 - recall: 0.9668 - val_loss: 0.1489 - val_accuracy: 0.0034 - val_precision: 0.9574 - val_recall: 0.9574
Epoch 6/60
13526/13526 [==============================] - 0s 16us/sample - loss: 0.0890 - accuracy: 0.0059 - precision: 0.9679 - recall: 0.9679 - val_loss: 0.1289 - val_accuracy: 0.0084 - val_precision: 0.9589 - val_recall: 0.9589
Epoch 7/60
13526/13526 [==============================] - 0s 18us/sample - loss: 0.0822 - accuracy: 0.0089 - precision: 0.9711 - recall: 0.9711 - val_loss: 0.1190 - val_accuracy: 0.0031 - val_precision: 0.9639 - val_recall: 0.9639
Epoch 8/60
13526/13526 [==============================] - 0s 17us/sample - loss: 0.0808 - accuracy: 0.0098 - precision: 0.9721 - recall: 0.9721 - val_loss: 0.1359 - val_accuracy: 0.0093 - val_precision: 0.9583 - val_recall: 0.9583
Epoch 9/60
13526/13526 [==============================] - 0s 20us/sample - l

In [13]:
"""""""""""""""""""""""
" BaseLine75
"""""""""""""""""""""""
results.append(executeNN(x[:int(len(x)*0.75)], y[:int(len(y)*0.75)], "NN/BaseLine75",0.005,60, [100,100,2]))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 100)               7900      
_________________________________________________________________
dense_29 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_30 (Dense)             (None, 2)                 202       
Total params: 18,202
Trainable params: 18,202
Non-trainable params: 0
_________________________________________________________________
Train on 10144 samples, validate on 2537 samples
Epoch 1/60
10144/10144 [==============================] - 1s 60us/sample - loss: 0.2034 - accuracy: 0.0017 - precision: 0.9264 - recall: 0.9264 - val_loss: 0.1585 - val_accuracy: 0.0020 - val_precision: 0.9448 - val_recall: 0.9448
Epoch 2/60
10144/10144 [==============================] - 0s 18us/sample - loss: 0.1328 - accuracy: 0.0016 - precision: 0.

Epoch 33/60
10144/10144 [==============================] - ETA: 0s - loss: 0.0386 - accuracy: 0.3445 - precision: 0.9907 - recall: 0.990 - 0s 23us/sample - loss: 0.0363 - accuracy: 0.3428 - precision: 0.9912 - recall: 0.9912 - val_loss: 0.2316 - val_accuracy: 0.3457 - val_precision: 0.9606 - val_recall: 0.9606
Epoch 34/60
10144/10144 [==============================] - 0s 24us/sample - loss: 0.0331 - accuracy: 0.3475 - precision: 0.9895 - recall: 0.9895 - val_loss: 0.2621 - val_accuracy: 0.3390 - val_precision: 0.9566 - val_recall: 0.9566
Epoch 35/60
10144/10144 [==============================] - 0s 24us/sample - loss: 0.0316 - accuracy: 0.3526 - precision: 0.9911 - recall: 0.9911 - val_loss: 0.2912 - val_accuracy: 0.3674 - val_precision: 0.9582 - val_recall: 0.9582
Epoch 36/60
10144/10144 [==============================] - 0s 21us/sample - loss: 0.0318 - accuracy: 0.3627 - precision: 0.9911 - recall: 0.9911 - val_loss: 0.2581 - val_accuracy: 0.3814 - val_precision: 0.9586 - val_recall:

In [14]:
"""""""""""""""""""""""
" BaseLine50
"""""""""""""""""""""""
results.append(executeNN(x[:int(len(x)*0.50)], y[:int(len(y)*0.50)], "NN/BaseLine50",0.005,60, [100,100,2]))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 100)               7900      
_________________________________________________________________
dense_32 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_33 (Dense)             (None, 2)                 202       
Total params: 18,202
Trainable params: 18,202
Non-trainable params: 0
_________________________________________________________________
Train on 6763 samples, validate on 1691 samples
Epoch 1/60
6763/6763 [==============================] - 1s 78us/sample - loss: 0.2278 - accuracy: 1.4786e-04 - precision: 0.9141 - recall: 0.9141 - val_loss: 0.1491 - val_accuracy: 0.0000e+00 - val_precision: 0.9462 - val_recall: 0.9462
Epoch 2/60
6763/6763 [==============================] - 0s 18us/sample - loss: 0.1459 - accuracy: 2.2180e-04 - precis

In [15]:
"""""""""""""""""""""""
" BaseLine25
"""""""""""""""""""""""
results.append(executeNN(x[:int(len(x)*0.25)], y[:int(len(y)*0.25)], "NN/BaseLine25",0.005,60, [100,100,2]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 100)               7900      
_________________________________________________________________
dense_35 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_36 (Dense)             (None, 2)                 202       
Total params: 18,202
Trainable params: 18,202
Non-trainable params: 0
_________________________________________________________________
Train on 3381 samples, validate on 846 samples
Epoch 1/60
3381/3381 [==============================] - 1s 157us/sample - loss: 0.2841 - accuracy: 2.9577e-04 - precision: 0.8991 - recall: 0.8991 - val_loss: 0.1717 - val_accuracy: 0.0012 - val_precision: 0.9326 - val_recall: 0.9326
Epoch 2/60
3381/3381 [==============================] - 0s 23us/sample - loss: 0.1668 - accuracy: 1.4789e-04 - precision:

In [18]:
display(pd.DataFrame(results).transpose())
pd.DataFrame(results).transpose().to_pickle("NNResult.json")


,0,1,2,3,4,5,6,7,8,9,10,11
run_name,NN/BaseLine,NN/CouchePlus,NN/CoucheMoins,NN/PerceptronPlus,NN/PerceptronMoins,NN/EpochPlus,NN/EpochMoins,NN/LearningRatePlus,NN/LearningRateMoins,NN/BaseLine75,NN/BaseLine50,NN/BaseLine25
train_time (seconds),13.5398,18.1906,11.9895,40.1523,13.3691,48.2944,6.86552,16.4337,17.8652,14.6324,11.6123,7.92001
learning rate,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.01,0.0025,0.005,0.005,0.005
epoch,60,60,60,60,60,200,20,60,60,60,60,60
accuracy,0.487579,0.778316,0.263308,0.577924,0.192407,0.786337,0.192555,0.522919,0.419451,0.494381,0.448026,0.40491
val_accuracy,0.48906,0.763601,0.266115,0.513601,0.189385,0.792578,0.203282,0.506801,0.408634,0.476153,0.445594,0.407801
f1,0.993494,0.988171,0.989502,0.988023,0.985362,0.998226,0.982626,0.987506,0.995194,0.99517,0.996008,0.996155
val_f1,0.959196,0.959196,0.95683,0.961266,0.958604,0.962448,0.956535,0.958013,0.957422,0.955459,0.959196,0.949173
layer: 0,100,100,100,500,20,100,100,100,100,100,100,100
layer: 1,100,100,2,500,20,100,100,100,100,100,100,100
